In [18]:
import numpy as np
from numba import njit

def sum_array(arr):
    total = 0.0
    for item in arr:
        total += item
    return total

def np_sum_array(arr):
    return np.sum(arr)

def np_arr_diff(arr):
    return arr - arr

def np_arr_np_diff(arr):
    return np.diff(arr)

def np_arr_product(arr):
    return 3 * arr

def np_arr_sqrt(arr):
    return np.sqrt(arr)

def np_arr_exp(arr):
    return np.exp(arr)

def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value


TEST_ARRAY = np.array(range(300))
test_funcs = [
    # These ones are faster in numba
    sum_array, monotonically_increasing, 
    # These ones are not
    np_sum_array, np_arr_diff, np_arr_np_diff,
    np_arr_product,
    np_arr_exp, np_arr_sqrt,
    ]

def timeit(fn, args):
    import time
    RUNS = 1000000
    fn(args)
    start_time = time.perf_counter()
    for _ in range(RUNS):
        fn(args)
    end_time  = time.perf_counter()
    print((end_time-start_time)/RUNS)

for fn in test_funcs:
    print(f'=============== Testing {fn.__name__} ===============')
    print(f'Non-jitted:')
    timeit(fn, TEST_ARRAY)
    fn = njit(fastmath=True, inline='always')(fn)
    print(f'Jitted:')
    timeit(fn, TEST_ARRAY)


=============== Testing sum_array ===============
Non-jitted:
2.568013855299796e-05
Jitted:
5.642856549966382e-07
=============== Testing np_sum_array ===============
Non-jitted:
2.221079902999918e-06
Jitted:
1.7378649499733e-07
=============== Testing np_arr_diff ===============
Non-jitted:
4.178987150080502e-07
Jitted:
5.944396979903104e-07
=============== Testing np_arr_np_diff ===============
Non-jitted:
2.018908740996267e-06
Jitted:
1.012284915006603e-06
=============== Testing np_arr_product ===============
Non-jitted:


## Summary of JIT Performance
1. `list` could lead to longer exeuction time in JIT optimization, than the non-jitted version. Use `np.array` instead.
2. JIT performs well in: 
    ```
    def np_sum_array(arr):
        return np.sum(arr)
    ```
    - Also performs well in Raw looping in python, where numpy cannot help

3. JIT does NOT perform well (i.e., slower than its non-jitted counter part) in:
    ```
    def np_arr_diff(arr):
        return arr - arr

    def np_arr_product(arr):
        return 3 * arr
    ```